# 確率ロボティクス2017第7回

上田隆一

2017年10月18日@千葉工業大学

## 今日やること

* 有限マルコフ決定過程

## 有限マルコフ決定過程

* finite Markov decision processes (finite MDPs, 有限MDP)
  * ロボットの行動を考えるときによく使う
    * 行動をするに当たって何が「最適」なのか？
    * 最適にするにはロボットがどうすべきか？
  * 昔ながらの（今も使われる）経路計画の定式化も有限MDPの部分問題
  
## 有限MDPの考え方

* ロボット（環境）の状態は制御出力により遷移
  * これまでの話と同じ
  * 雑音が伴うのも同じ
* 状態をエージェントは観察できる
  * これまでとは違う
* 「こういう状態になったら終わり」という状態が存在
  * 「終端状態」 
  * 例
    * 移動ロボットが目的地に入った状態
    * ロボットが川に落ちた
* 状態遷移にはコストやペナルティーが伴う
  * コスト: 時間、電力消費、燃料消費等
  * ペナルティー: ルール違反、危険な行為に対して
  
## 有限MDPの定式化

### 状態

* 離散状態の集合を考える
  * $\mathcal{S}= \{s^{(i)} | i=1,2,\dots,N \}$
* 状態が連続の場合は離散化
  * $\mathcal{X}$を区切る 

### 終端状態

* $s_\text{f} \in \mathcal{S}_\text{f} \subset \mathcal{S}$
* この状態に到達するとそれ以上時間が進まない
* タスクの終わり 

### 制御出力（行動）

* 有限個の行動の集合から選択
  * $\mathcal{A} = \{ a^{(j)} | j = 1,2,\dots,M \}$
* 連続系の$\mathcal{U}$から何種類か選んで有限個に
* この資料では$s_i,a_j$と書くときは時刻を表すことにします

### 状態遷移

* $s' \sim P(s' | s,a)$ 
  * $s,s'$はそれぞれ遷移前、遷移後の状態を示す
* $P(s' | s,a)$を$\mathcal{P}^a_{ss'}$と表記しましょう
  * $s \in \mathcal{S} - \mathcal{S}_\text{f}$から$a \in \mathcal{A}$を選択した時、$s' \in \mathcal{S}$に遷移する確率
  
### 状態遷移の評価

* $R(s,a,s') \in \Re$
  * 評価は常に実数で行われる
* 時刻に依存しないと仮定しましょう
* 評価軸がいくつあっても実数に置き換えて一元化
  * 例: 移動ロボットが水たまりを踏んだら1回につき3分のペナルティー、等
* $R(s,a,s')$を$\mathcal{R}^a_{ss'}$と表記

### 終端状態の評価

* どの終端状態に行くかも評価の対象
* 終端状態の価値
  * $V(s)\in \Re$ where $\forall s \in \mathcal{S}_\text{f}$
  * この値も$\mathcal{R}^a_{ss'}$と同じ評価軸の上にある
  
### 行動の評価

* ある状態からスタートして、ある終端状態に入った時の評価の和
* 状態遷移と行動が$s_0,a_1,s_1,a_2,\dots,s_T$だったときの評価:
  * $J(s_{0:T},a_{1:T})$ $ = \mathcal{R}^{a_1}_{s_0s_1} +\mathcal{R}^{a_2}_{s_1s_2} +\mathcal{R}^{a_3}_{s_2s_3} + \dots +\mathcal{R}^{a_T}_{s_{T-1}s_T} + V(s_T)$
$ = \sum_{t=1}^T \mathcal{R}^{a_t}_{s_{t-1}s_t} + V(s_T)  $
* $J(s_{0:T},a_{1:T})$は毎回違う
  * $\mathcal{P}^a_{ss'}$で、事後の状態$s'$がばらつく
  
### 有限MDPでの最適な行動

* $J(s_{0:T},a_{1:T})$の期待値が最大になる行動決定則
  * 毎回、事後状態がばらつくのにどうやって求めるの？
  * 結構難しい
    * 識別の問題なら判断は1回
    * 行動決定の場合は終端状態に至るまで何度も判断
    

### 例題

次のような簡単な問題を扱います。

3x3の碁盤の世界を準備します。ロボットはマス目のどこかに存在し、1歩で上下左右に1マス移動できます。ただし、10%の確率で移動に失敗し、元のマス目に戻ります。右上をゴールにして、あるマス目をスタート地点とするとき、何歩でゴールに達するかを、全マス目に対して計算しましょう。ただし、この世界は壁に囲まれていて、壁を超えて移動しようとすると元のマスに戻されます。


#### 準備

次のような変数を準備します。

* costs: 各マスをスタートにしたときのゴールまでの歩数を記録する2次元のリスト
* goal: ゴールの座標。ゴールは2つ以上でも良いが、この問題では1個だけ
* actions: ロボットが選択できる行動のリスト

valuesの初期値はデタラメで大丈夫ですが、ゴールに相当するマス目のものだけ正解の0にセットしておきます。

    
## 状態のマルコフ性と最適性の原理

* $\mathcal{P}^a_{ss'}, \mathcal{R}^a_{ss'}$が時不変
  * 違う時刻に同じ状態に来たら、同じ行動をとれば同じ確率分布で次の状態に遷移し、同じように評価される
* これが成り立つと・・・
  * ある試行の状態遷移$s_0,s_1,s_2,\dots,s_N$のうち、どこを初期状態にしても、$J(s_{0:T},a_{1:T})$の期待値を最大にする問題は、同じ問題になる。
  
## 状態の価値

* $J(s_{0:T},a_{1:T})$の期待値の最大値を考えてみる 
  * 先ほどの議論から、$J(s_{0:T},a_{1:T})$の期待値の最大値が存在すると仮定すると、初期状態だろうがなかろうが、ある状態$s$から先の行動決定を考えると、この期待値の最大値が存在するし、それは、その状態$s$の関数となる
  * これをその状態の（エージェントが最適な行動を取った時の）「価値」と言う
* 終端状態の価値$V(s) ( s \in \mathcal{S}_\text{f})$を拡張して、状態空間全体の関数: $V^*: \mathcal{S} \to \Re$とする
  * $V^*$: 最適状態価値関数 

## 価値の性質（ベルマン方程式）

* $V^* (s) = \max_{a\in\mathcal{A}} \sum_{s' \in \mathcal{S}} \mathcal{P}_{ss'}^a \left[ V^*(s') + \mathcal{R}_{ss'}^a \right] $
  * もし最適状態価値関数が存在するなら、ある状態$\forall s \in \mathcal{S}-\mathcal{S}_\text{f}$の値$V^*(s)$は、以下の値を最大にする行動$a \in \mathcal{A}$を選んだ時の値になる
    * $s$から遷移した先の値$V^*(s')$と、その状態遷移に対する評価$\mathcal{R}_{ss'}^a$を足した値を考える。遷移先は確率的にばらつくので、この値について$\mathcal{P}_{ss'}^a$で重み付き平均を求めたもの。

## 価値の計算（価値反復）

* ある状態$s$の$V^*(s)$だけ分からない。あとの状態の$V^*$の値は分かるという、都合の良い>状況を考えてみましょう。
  * 価値の計算式はベルマン方程式そのものとなる
    * $V^* (s) =\max_{a\in\mathcal{A}} \sum_{s' \in \mathcal{S}} \mathcal{P}_{ss'}^a \left[ V^*(s') + \mathcal{R}_{ss'}^a \right] $
* 価値反復アルゴリズム
  * 終端状態の価値以外を適当に初期化して、上の計算を$\forall s \in \mathcal{S} - \mathcal{S}_\text{f}$に対して何度も計算していく
  * 終端状態に近い状態から$V^* (s)$の値が収束していく 

## 最適方策

* 最適状態価値関数が分かっていると、ベルマン方程式を成立させるための行動が求まる
  * 最適な行動: $\text{argmax}_a \sum_{s' \in \mathcal{S}} \mathcal{P}_{ss'}^a \left[ V^*(s') + \mathcal{R}_{ss'}^a \right] $
* 「最適方策」
  * $\pi^*: \mathcal{S}-\mathcal{S}_\text{f} \to \mathcal{A}$
    * 最適方策は状態と行動を対にしたリストとなる
    * 時刻は全く関係ない
    * $\pi^*$を知っているロボットは、置き直されたりタスクを妨害されたりしても、その後に自身の状態を知覚できれば、最適な行動をとることが可能